# RNN ( many to on stacked )
> 참고
1. https://github.com/deeplearningzerotoall/TensorFlow/blob/master/tf_2.x/lab-12-2-many-to-one-stacking-keras-eager.ipynb

* RNN의 Many to One형태를 여러겹 쌓아서 (Stacked ) Binary 분류를 하는 예제

In [1]:
import tensorflow as tf

## Data & Pre-Processing -1 

In [2]:
data = ['What I cannot create, I do not understand.',
             'Intellecuals solve problems, geniuses prevent them',
             'A person who never made a mistake never tied anything new.',
             'The same equations have the same solution.']
target = [1, 0, 0, 1]

# token dictionary
char_set = ['<pad>']+sorted(list(set("".join(data))))
idx_to_char = {idx : char for idx, char in enumerate(char_set)}
char_to_idx = {char : idx for idx, char in enumerate(char_set)}

print(char_set)
print(idx_to_char)
print(char_to_idx)

['<pad>', ' ', ',', '.', 'A', 'I', 'T', 'W', 'a', 'b', 'c', 'd', 'e', 'g', 'h', 'i', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'y']
{0: '<pad>', 1: ' ', 2: ',', 3: '.', 4: 'A', 5: 'I', 6: 'T', 7: 'W', 8: 'a', 9: 'b', 10: 'c', 11: 'd', 12: 'e', 13: 'g', 14: 'h', 15: 'i', 16: 'k', 17: 'l', 18: 'm', 19: 'n', 20: 'o', 21: 'p', 22: 'q', 23: 'r', 24: 's', 25: 't', 26: 'u', 27: 'v', 28: 'w', 29: 'y'}
{'<pad>': 0, ' ': 1, ',': 2, '.': 3, 'A': 4, 'I': 5, 'T': 6, 'W': 7, 'a': 8, 'b': 9, 'c': 10, 'd': 11, 'e': 12, 'g': 13, 'h': 14, 'i': 15, 'k': 16, 'l': 17, 'm': 18, 'n': 19, 'o': 20, 'p': 21, 'q': 22, 'r': 23, 's': 24, 't': 25, 'u': 26, 'v': 27, 'w': 28, 'y': 29}


## Data & Pre-Processing - 2

In [5]:
# token -> sequence
data_ = list(map(lambda sentence : [char_to_idx.get(char) for char in sentence], data))
data_X_len = list(map(lambda sentence : len(sentence), data))

# padding
max_sequece = 55
data_X = tf.keras.preprocessing.sequence.pad_sequences(sequences=data_x, maxlen=max_sequece,
                                                       padding='post', truncating='post')
print(data_X)
print(data_X_len)

[[ 7 14  8 25  1  5  1 10  8 19 19 20 25  1 10 23 12  8 25 12  2  1  5  1
  11 20  1 19 20 25  1 26 19 11 12 23 24 25  8 19 11  3  0  0  0  0  0  0
   0  0  0  0  0  0  0]
 [ 5 19 25 12 17 17 12 10 26  8 17 24  1 24 20 17 27 12  1 21 23 20  9 17
  12 18 24  2  1 13 12 19 15 26 24 12 24  1 21 23 12 27 12 19 25  1 25 14
  12 18  0  0  0  0  0]
 [ 4  1 21 12 23 24 20 19  1 28 14 20  1 19 12 27 12 23  1 18  8 11 12  1
   8  1 18 15 24 25  8 16 12  1 19 12 27 12 23  1 25 15 12 11  1  8 19 29
  25 14 15 19 13  1 19]
 [ 6 14 12  1 24  8 18 12  1 12 22 26  8 25 15 20 19 24  1 14  8 27 12  1
  25 14 12  1 24  8 18 12  1 24 20 17 26 25 15 20 19  3  0  0  0  0  0  0
   0  0  0  0  0  0  0]]
[42, 50, 58, 42]


## Dataset

In [7]:
batch_size = 2

# Dataset
dataset = tf.data.Dataset.from_tensor_slices((data_X, target))
dataset = dataset.shuffle(buffer_size=4).batch(batch_size=batch_size)

print(dataset)

<BatchDataset shapes: ((None, 55), (None,)), types: (tf.int32, tf.int32)>


## Parameter

In [8]:
# Parameter
num_classes = 2
hidden_dims = [10, 10]
input_dim = len(char_to_idx)
output_dim = len(char_to_idx)

# hyper parameter
learning_rate = 0.01
epochs = 30
optimizer = tf.optimizers.Adam(learning_rate=learning_rate)

## Model

In [9]:
# Model
model = tf.keras.Sequential()
model.add(tf.keras.layers.Embedding(input_dim=input_dim, output_dim=output_dim,
                                    trainable=False, mask_zero=True, input_length=max_sequece))
cell1 = tf.keras.layers.SimpleRNNCell(units=hidden_dims[0])
model.add(tf.keras.layers.RNN(cell=cell1, return_sequences=True))
model.add(tf.keras.layers.TimeDistributed(tf.keras.layers.Dropout(rate = 0.2)))
cell2 = tf.keras.layers.SimpleRNNCell(units=hidden_dims[1])
model.add(tf.keras.layers.RNN(cell=cell2))
model.add(tf.keras.layers.Dropout(rate = 0.2))
model.add(tf.keras.layers.Dense(units=num_classes))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 55, 30)            900       
_________________________________________________________________
rnn (RNN)                    (None, 55, 10)            410       
_________________________________________________________________
time_distributed (TimeDistri (None, 55, 10)            0         
_________________________________________________________________
rnn_1 (RNN)                  (None, 10)                210       
_________________________________________________________________
dropout_1 (Dropout)          (None, 10)                0         
_________________________________________________________________
dense (Dense)                (None, 2)                 22        
Total params: 1,542
Trainable params: 642
Non-trainable params: 900
______________________________________________________

## Function

In [10]:
# cost_function
def cost_function(model, x, y, training):
    return tf.reduce_mean(tf.keras.losses.sparse_categorical_crossentropy(
        y_true=y, y_pred=model(x, training), from_logits=True))

## Training

In [12]:
cost_log = []
for epoch in range(epochs):
    avg_cost = 0
    step = 0

    for x_batch, y_batch in dataset:
        with tf.GradientTape() as tape:
            cost = cost_function(model, x_batch, y_batch, training=True)
        grads = tape.gradient(cost, model.variables)
        optimizer.apply_gradients(grads_and_vars=zip(grads, model.variables))
        avg_cost += cost
        step += 1

    avg_cost /= step
    cost_log.append(avg_cost)

    if (epoch + 1) % 5 == 0:
        print("epoch : {:3}, cost : {:.3f}".format(epoch + 1, avg_cost))
        
hypothesis = tf.argmax(model(data_X), axis=-1)
print(tf.reduce_mean(tf.cast(tf.equal(hypothesis, target),dtype=tf.float32)))

epoch :   5, cost : 0.001
epoch :  10, cost : 0.001
epoch :  15, cost : 0.007
epoch :  20, cost : 0.001
epoch :  25, cost : 0.003
epoch :  30, cost : 0.001
tf.Tensor(1.0, shape=(), dtype=float32)
